In [2]:
from FlagEmbedding import FlagModel
model = FlagModel('BAAI/bge-large-en-v1.5', 
                  query_instruction_for_retrieval="",
                  use_fp16=True)

----------using 8*GPUs----------


In [3]:
from datasets import load_dataset, load_from_disk

In [39]:
root = "/mnt/bn/data-tns-live-llm/leon/datasets/fed_data/"
datas = []
for i in range(10):
    datas.append(load_from_disk(f"{root}/pos_{i}.parquet"))

In [40]:
print(datas[0])

Dataset({
    features: ['instruction', 'response', '__index_level_0__'],
    num_rows: 5100
})


In [41]:
def concat_columns(example: dict):
    example['text'] = example['instruction'] + ' ' + example['response']
    return example

In [42]:
for i in range(10):
    datas[i] = datas[i].map(concat_columns)

In [45]:
print(datas[0]["text"][:10])

['At what level do the psoas muscle and iliacus muscle combine to form the iliopsoas muscle? The psoas muscle combines with the iliacus muscle around the level of the inguinal ligament to form the iliopsoas muscle.', "a shopkeeper sells 20 % of his stock at 10 % profit ans sells the remaining at a loss of 5 % . he incurred an overall loss of rs . 350 . find the total worth of the stock ? Let's write a Python program. n0 = 20.0\nn1 = 10.0\nn2 = 5.0\nn3 = 350.0\nt0 = n2 / 100.0\nt1 = n1 / 100.0\nt2 = n0 / 100.0\nt3 = 100.0 - n0\nt4 = t3 / 100.0\nt5 = t1 * t2\nt6 = t0 * t4\nt7 = t6 - t5\nanswer = n3 / t7\nprint(answer)", "Create a title for the given book summary. John is an accountant who discovers he can time travel after getting into an accident. Journey Through Time: A Accountant's Incredible Adventure.", 'What is a possible clinical manifestation of Friedreich ataxia? Friedreich ataxia may present with diabetes.', "In Kiran's opinion, his weight is greater than 65 kg but less than 72

In [24]:
def concat_columns(example: dict):
    # 将 instruction 和 input 列的值拼接起来，用空格分隔
    if "input" in example.keys() and example["input"] is not None:
        example['text'] = example['instruction'] + ' ' + example['input'] + example["output"]
    else:
        example["text"] = example["instruction"] + example["output"]
    return example

In [28]:
data1 = load_from_disk("/mnt/bn/data-tns-live-llm/leon/datasets/pos.parquet").map(concat_columns)
data2 = load_from_disk("/mnt/bn/data-tns-live-llm/leon/datasets/neg.parquet").map(concat_columns)

In [29]:
from pprint import pprint as pp
pp(data1["text"][:5])
pp(data2["text"][:5])

['Write a valid JavaScript statement to display the given text. Hello '
 'World!console.log("Hello World!");',
 'Create an HTML document to display "Hello, World!" in bold. <!DOCTYPE html>\n'
 '<html>\n'
 '  <head>\n'
 '    <title>Hello World</title>\n'
 '  </head>\n'
 '  <body>\n'
 '    <b>Hello, World!</b>    \n'
 '  </body>\n'
 '</html>',
 'Write a HTML code for a webpage that displays the headline "Hello World!". '
 '<html>\n'
 ' <head>\n'
 '  <title>Hello World</title>\n'
 ' </head>\n'
 ' <body>\n'
 '  <h1>Hello World!</h1>\n'
 ' </body>\n'
 '</html>',
 'Create the HTML code necessary to display the following text. <html>\n'
 '    <head>\n'
 '    </head>\n'
 '    <body>\n'
 '        <p>Hello, world!</p>\n'
 '    </body>\n'
 '</html>',
 'Write a JavaScript code snippet to display a welcome message to the user. '
 "console.log('Welcome!');"]
['How did the Black Plague change Europe? The Black Plague had a major impact '
 'on Europe, resulting in a decline in population, widespread e